In [1]:
import utils.helpers as helpers
from utils.paths import *


import SimpleITK  # conda install -c https://conda.anaconda.org/simpleitk SimpleITK
import numpy
import pandas
import numpy as np
import pandas as pd
import ntpath
import cv2  # conda install -c https://conda.anaconda.org/menpo opencv3
import shutil
import random
import math
import multiprocessing
import os
import glob
import h5py
from joblib import Parallel, delayed
from tqdm import tqdm
import matplotlib.pyplot as plt
import skimage
import sklearn
import time

from typing import List, Tuple
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter
from skimage.transform import resize
from sklearn.utils import shuffle

from keras.optimizers import Adam, SGD
from keras.layers import Input, Convolution2D, MaxPooling2D, UpSampling2D, merge, Dropout, BatchNormalization,SpatialDropout2D,Convolution3D,MaxPooling3D, UpSampling3D, Flatten, Dense
from keras.models import Model, load_model, model_from_json
from keras.metrics import binary_accuracy, binary_crossentropy, mean_squared_error, mean_absolute_error
from keras import backend as K
from keras.callbacks import ModelCheckpoint, Callback, LearningRateScheduler,EarlyStopping
from keras.backend.tensorflow_backend import set_session
from keras.utils.vis_utils import plot_model
from keras.initializers import glorot_normal, he_uniform 

random.seed(1321)
numpy.random.seed(1321)

import warnings
warnings.filterwarnings("ignore")

K.set_image_dim_ordering('th')

Using TensorFlow backend.


## 训练模型

In [2]:
src = PATH['model_train']
model_paths = PATH['model_paths']

In [3]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

def dice_coef_np(y_true,y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return ((2. * intersection + 1) / (np.sum(y_true_f) + np.sum(y_pred_f) + 1))


def unet_model(dropout_rate,learn_rate, width):
    inputs = Input((1, 512, 512))
    conv1 = Convolution2D(width, (3, 3), padding="same", activation="relu")(inputs)
    conv1 = BatchNormalization(axis = 1)(conv1)
    conv1 = Convolution2D(width, (3, 3), padding="same", activation="relu")(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Convolution2D(width*2, (3, 3), padding="same", activation="relu")(pool1)
    conv2 = BatchNormalization(axis = 1)(conv2)
    conv2 = Convolution2D(width*2, (3, 3), padding="same", activation="relu")(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Convolution2D(width*4, (3, 3), padding="same", activation="relu")(pool2)
    conv3 = BatchNormalization(axis = 1)(conv3)
    conv3 = Convolution2D(width*4, (3, 3), padding="same", activation="relu")(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Convolution2D(width*8, (3, 3), padding="same", activation="relu")(pool3)
    conv4 = BatchNormalization(axis = 1)(conv4)
    conv4 = Convolution2D(width*8, (3, 3), padding="same", activation="relu")(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Convolution2D(width*16, (3, 3), padding="same", activation="relu")(pool4)
    conv5 = BatchNormalization(axis = 1)(conv5)
    conv5 = Convolution2D(width*16, (3, 3), padding="same", activation="relu")(conv5)

    up6 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=1)
    conv6 = SpatialDropout2D(dropout_rate)(up6)
    conv6 = Convolution2D(width*8, (3, 3), padding="same", activation="relu")(conv6)
    conv6 = Convolution2D(width*8, (3, 3), padding="same", activation="relu")(conv6)

    up7 = merge([UpSampling2D(size=(2, 2))(conv6), conv3], mode='concat', concat_axis=1)
    conv7 = SpatialDropout2D(dropout_rate)(up7)
    conv7 = Convolution2D(width*4, (3, 3), padding="same", activation="relu")(conv7)
    conv7 = Convolution2D(width*4, (3, 3), padding="same", activation="relu")(conv7)

    up8 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=1)
    conv8 = SpatialDropout2D(dropout_rate)(up8)
    conv8 = Convolution2D(width*2, (3, 3), padding="same", activation="relu")(conv8)
    conv8 = Convolution2D(width*2, (3, 3), padding="same", activation="relu")(conv8)

    up9 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=1)
    conv9 = SpatialDropout2D(dropout_rate)(up9)
    conv9 = Convolution2D(width, (3, 3), padding="same", activation="relu")(conv9)
    conv9 = Convolution2D(width, (3, 3), padding="same", activation="relu")(conv9)
    conv10 = Convolution2D(1, (1, 1), activation="sigmoid")(conv9)

    model = Model(input=inputs, output=conv10)
    #model.summary()
    #model.compile(optimizer=Adam(lr=learn_rate), loss=dice_coef_loss, metrics=[dice_coef])
    model.compile(optimizer=SGD(lr=learn_rate, momentum=0.9, nesterov=True), loss=dice_coef_loss, metrics=[dice_coef])
    
    #plot_model(model, to_file='model1.png',show_shapes=True)
    return model



def unet_fit(name, check_name = None):
    data_gen_args = dict(rotation_range=90.,   
                     width_shift_range=0.3,  
                     height_shift_range=0.3,   
                     horizontal_flip=True,   
                     vertical_flip=True,  
                     )
    from keras.preprocessing.image import ImageDataGenerator
    image_datagen = ImageDataGenerator(**data_gen_args)
    mask_datagen = ImageDataGenerator(**data_gen_args)

    # Provide the same seed and keyword arguments to the fit and flow methods
    seed = 1
    
    image_generator = image_datagen.flow_from_directory(
        src,
        class_mode=None,
        classes=['lung'],
        seed=seed,
        target_size=(512,512),
        color_mode="grayscale",
        batch_size=1)

    mask_generator = mask_datagen.flow_from_directory(
        src,
        class_mode=None,
        classes=['nodule'],
        seed=seed,
        target_size=(512,512),
        color_mode="grayscale",
        batch_size=1)

    # combine generators into one which yields image and masks
    train_generator = zip(image_generator, mask_generator)
        
    t = time.time()
    callbacks = [EarlyStopping(monitor='val_loss', patience = 200, 
                                   verbose = 1),
    ModelCheckpoint(model_paths + '{}.h5'.format(name), 
                        monitor='val_loss', 
                        verbose = 0, save_best_only = True)]
    if check_name is not None:
        check_model = model_paths + '{}.h5'.format(check_name)
        model = load_model(check_model, 
                           custom_objects={'dice_coef_loss': dice_coef_loss, 'dice_coef': dice_coef})
    else:
        model = unet_model(dropout_rate = 0.2, learn_rate = 1e-4, width = 32)   
        #model = get_unet(1e-5)        
        
    model.fit_generator(
        train_generator,
        epochs=1000,
        verbose =2, 
        callbacks = callbacks,
        steps_per_epoch=256,
        validation_data = train_generator,
        nb_val_samples = 48)
    return

In [4]:
unet_fit('final_fenge_170629_2')
#unet_fit('final_fenge_170629_2','final_fenge_170629_2')

Found 3711 images belonging to 1 classes.
Found 3711 images belonging to 1 classes.
Epoch 1/1000
55s - loss: -3.1030e-01 - dice_coef: 0.3103 - val_loss: -3.6029e-01 - val_dice_coef: 0.3603
Epoch 2/1000
50s - loss: -3.8991e-01 - dice_coef: 0.3899 - val_loss: -3.3442e-01 - val_dice_coef: 0.3344
Epoch 3/1000
51s - loss: -3.7897e-01 - dice_coef: 0.3790 - val_loss: -4.7217e-01 - val_dice_coef: 0.4722
Epoch 4/1000
50s - loss: -4.1107e-01 - dice_coef: 0.4111 - val_loss: -4.6475e-01 - val_dice_coef: 0.4648
Epoch 5/1000
51s - loss: -4.5459e-01 - dice_coef: 0.4546 - val_loss: -5.8809e-01 - val_dice_coef: 0.5881
Epoch 6/1000
51s - loss: -5.8230e-01 - dice_coef: 0.5823 - val_loss: -6.1902e-01 - val_dice_coef: 0.6190
Epoch 7/1000
51s - loss: -8.0677e-01 - dice_coef: 0.8068 - val_loss: -8.2513e-01 - val_dice_coef: 0.8251
Epoch 8/1000
51s - loss: -8.7241e-01 - dice_coef: 0.8724 - val_loss: -8.2989e-01 - val_dice_coef: 0.8299
Epoch 9/1000
51s - loss: -9.6132e-01 - dice_coef: 0.9613 - val_loss: -9.9294

Epoch 78/1000
50s - loss: -1.5079e+00 - dice_coef: 1.5079 - val_loss: -1.4235e+00 - val_dice_coef: 1.4235
Epoch 79/1000
51s - loss: -1.5963e+00 - dice_coef: 1.5963 - val_loss: -1.6851e+00 - val_dice_coef: 1.6851
Epoch 80/1000
50s - loss: -1.4955e+00 - dice_coef: 1.4955 - val_loss: -1.3057e+00 - val_dice_coef: 1.3057
Epoch 81/1000
50s - loss: -1.5494e+00 - dice_coef: 1.5494 - val_loss: -1.4496e+00 - val_dice_coef: 1.4496
Epoch 82/1000
50s - loss: -1.4835e+00 - dice_coef: 1.4835 - val_loss: -1.4218e+00 - val_dice_coef: 1.4218
Epoch 83/1000
50s - loss: -1.5059e+00 - dice_coef: 1.5059 - val_loss: -1.5364e+00 - val_dice_coef: 1.5364
Epoch 84/1000
50s - loss: -1.4763e+00 - dice_coef: 1.4763 - val_loss: -1.4969e+00 - val_dice_coef: 1.4969
Epoch 85/1000
50s - loss: -1.5989e+00 - dice_coef: 1.5989 - val_loss: -1.2909e+00 - val_dice_coef: 1.2909
Epoch 86/1000
50s - loss: -1.4720e+00 - dice_coef: 1.4720 - val_loss: -1.2543e+00 - val_dice_coef: 1.2543
Epoch 87/1000
50s - loss: -1.5566e+00 - dice_c

Epoch 155/1000
50s - loss: -1.6432e+00 - dice_coef: 1.6432 - val_loss: -1.2870e+00 - val_dice_coef: 1.2870
Epoch 156/1000
50s - loss: -1.5483e+00 - dice_coef: 1.5483 - val_loss: -1.3077e+00 - val_dice_coef: 1.3077
Epoch 157/1000
50s - loss: -1.5216e+00 - dice_coef: 1.5216 - val_loss: -1.4966e+00 - val_dice_coef: 1.4966
Epoch 158/1000
50s - loss: -1.6109e+00 - dice_coef: 1.6109 - val_loss: -1.4912e+00 - val_dice_coef: 1.4912
Epoch 159/1000
50s - loss: -1.6388e+00 - dice_coef: 1.6388 - val_loss: -1.5696e+00 - val_dice_coef: 1.5696
Epoch 160/1000
50s - loss: -1.5967e+00 - dice_coef: 1.5967 - val_loss: -1.5364e+00 - val_dice_coef: 1.5364
Epoch 161/1000
50s - loss: -1.6094e+00 - dice_coef: 1.6094 - val_loss: -1.6371e+00 - val_dice_coef: 1.6371
Epoch 162/1000
50s - loss: -1.5424e+00 - dice_coef: 1.5424 - val_loss: -1.4272e+00 - val_dice_coef: 1.4272
Epoch 163/1000
50s - loss: -1.5288e+00 - dice_coef: 1.5288 - val_loss: -1.3889e+00 - val_dice_coef: 1.3889
Epoch 164/1000
50s - loss: -1.5328e+0

Epoch 232/1000
50s - loss: -1.6142e+00 - dice_coef: 1.6142 - val_loss: -1.4360e+00 - val_dice_coef: 1.4360
Epoch 233/1000
50s - loss: -1.6213e+00 - dice_coef: 1.6213 - val_loss: -1.4153e+00 - val_dice_coef: 1.4153
Epoch 234/1000
50s - loss: -1.6150e+00 - dice_coef: 1.6150 - val_loss: -1.4499e+00 - val_dice_coef: 1.4499
Epoch 235/1000
50s - loss: -1.5144e+00 - dice_coef: 1.5144 - val_loss: -1.4985e+00 - val_dice_coef: 1.4985
Epoch 236/1000
50s - loss: -1.7038e+00 - dice_coef: 1.7038 - val_loss: -1.5421e+00 - val_dice_coef: 1.5421
Epoch 237/1000
50s - loss: -1.6033e+00 - dice_coef: 1.6033 - val_loss: -1.5883e+00 - val_dice_coef: 1.5883
Epoch 238/1000
50s - loss: -1.6495e+00 - dice_coef: 1.6495 - val_loss: -1.5118e+00 - val_dice_coef: 1.5118
Epoch 239/1000
50s - loss: -1.6248e+00 - dice_coef: 1.6248 - val_loss: -1.5254e+00 - val_dice_coef: 1.5254
Epoch 240/1000
50s - loss: -1.6545e+00 - dice_coef: 1.6545 - val_loss: -1.5846e+00 - val_dice_coef: 1.5846
Epoch 241/1000
50s - loss: -1.6315e+0

Epoch 309/1000
50s - loss: -1.6879e+00 - dice_coef: 1.6879 - val_loss: -1.4354e+00 - val_dice_coef: 1.4354
Epoch 310/1000
50s - loss: -1.6580e+00 - dice_coef: 1.6580 - val_loss: -1.6594e+00 - val_dice_coef: 1.6594
Epoch 311/1000
50s - loss: -1.5746e+00 - dice_coef: 1.5746 - val_loss: -1.3768e+00 - val_dice_coef: 1.3768
Epoch 312/1000
50s - loss: -1.5806e+00 - dice_coef: 1.5806 - val_loss: -1.7011e+00 - val_dice_coef: 1.7011
Epoch 313/1000
50s - loss: -1.6617e+00 - dice_coef: 1.6617 - val_loss: -1.5393e+00 - val_dice_coef: 1.5393
Epoch 314/1000
50s - loss: -1.6223e+00 - dice_coef: 1.6223 - val_loss: -1.6231e+00 - val_dice_coef: 1.6231
Epoch 315/1000
50s - loss: -1.5835e+00 - dice_coef: 1.5835 - val_loss: -1.4427e+00 - val_dice_coef: 1.4427
Epoch 316/1000
50s - loss: -1.6669e+00 - dice_coef: 1.6669 - val_loss: -1.4390e+00 - val_dice_coef: 1.4390
Epoch 317/1000
50s - loss: -1.6518e+00 - dice_coef: 1.6518 - val_loss: -1.3936e+00 - val_dice_coef: 1.3936
Epoch 318/1000
50s - loss: -1.5842e+0

Epoch 386/1000
50s - loss: -1.6277e+00 - dice_coef: 1.6277 - val_loss: -1.6264e+00 - val_dice_coef: 1.6264
Epoch 387/1000
50s - loss: -1.7050e+00 - dice_coef: 1.7050 - val_loss: -1.5375e+00 - val_dice_coef: 1.5375
Epoch 388/1000
50s - loss: -1.6160e+00 - dice_coef: 1.6160 - val_loss: -1.5575e+00 - val_dice_coef: 1.5575
Epoch 389/1000
50s - loss: -1.5813e+00 - dice_coef: 1.5813 - val_loss: -1.5508e+00 - val_dice_coef: 1.5508
Epoch 390/1000
50s - loss: -1.6708e+00 - dice_coef: 1.6708 - val_loss: -1.6115e+00 - val_dice_coef: 1.6115
Epoch 391/1000
50s - loss: -1.5755e+00 - dice_coef: 1.5755 - val_loss: -1.3478e+00 - val_dice_coef: 1.3478
Epoch 392/1000
50s - loss: -1.6273e+00 - dice_coef: 1.6273 - val_loss: -1.4062e+00 - val_dice_coef: 1.4062
Epoch 393/1000
50s - loss: -1.7177e+00 - dice_coef: 1.7177 - val_loss: -1.6258e+00 - val_dice_coef: 1.6258
Epoch 394/1000
51s - loss: -1.6133e+00 - dice_coef: 1.6133 - val_loss: -1.7886e+00 - val_dice_coef: 1.7886
Epoch 395/1000
50s - loss: -1.6362e+0

Epoch 463/1000
50s - loss: -1.6124e+00 - dice_coef: 1.6124 - val_loss: -1.6279e+00 - val_dice_coef: 1.6279
Epoch 464/1000
50s - loss: -1.5871e+00 - dice_coef: 1.5871 - val_loss: -1.4417e+00 - val_dice_coef: 1.4417
Epoch 465/1000
50s - loss: -1.5956e+00 - dice_coef: 1.5956 - val_loss: -1.4169e+00 - val_dice_coef: 1.4169
Epoch 466/1000
50s - loss: -1.6026e+00 - dice_coef: 1.6026 - val_loss: -1.4099e+00 - val_dice_coef: 1.4099
Epoch 467/1000
50s - loss: -1.6165e+00 - dice_coef: 1.6165 - val_loss: -1.4775e+00 - val_dice_coef: 1.4775
Epoch 468/1000
50s - loss: -1.6593e+00 - dice_coef: 1.6593 - val_loss: -1.4628e+00 - val_dice_coef: 1.4628
Epoch 469/1000
50s - loss: -1.5757e+00 - dice_coef: 1.5757 - val_loss: -1.5293e+00 - val_dice_coef: 1.5293
Epoch 470/1000
50s - loss: -1.5897e+00 - dice_coef: 1.5897 - val_loss: -1.3910e+00 - val_dice_coef: 1.3910
Epoch 471/1000
50s - loss: -1.5953e+00 - dice_coef: 1.5953 - val_loss: -1.5500e+00 - val_dice_coef: 1.5500
Epoch 472/1000
50s - loss: -1.6382e+0

Epoch 540/1000
50s - loss: -1.6370e+00 - dice_coef: 1.6370 - val_loss: -1.6136e+00 - val_dice_coef: 1.6136
Epoch 541/1000
50s - loss: -1.6429e+00 - dice_coef: 1.6429 - val_loss: -1.6254e+00 - val_dice_coef: 1.6254
Epoch 542/1000
50s - loss: -1.6222e+00 - dice_coef: 1.6222 - val_loss: -1.5573e+00 - val_dice_coef: 1.5573
Epoch 543/1000
50s - loss: -1.6456e+00 - dice_coef: 1.6456 - val_loss: -1.7861e+00 - val_dice_coef: 1.7861
Epoch 544/1000
50s - loss: -1.6504e+00 - dice_coef: 1.6504 - val_loss: -1.5859e+00 - val_dice_coef: 1.5859
Epoch 545/1000
50s - loss: -1.6383e+00 - dice_coef: 1.6383 - val_loss: -1.4678e+00 - val_dice_coef: 1.4678
Epoch 546/1000
50s - loss: -1.6766e+00 - dice_coef: 1.6766 - val_loss: -1.5335e+00 - val_dice_coef: 1.5335
Epoch 547/1000
50s - loss: -1.6620e+00 - dice_coef: 1.6620 - val_loss: -1.4261e+00 - val_dice_coef: 1.4261
Epoch 548/1000
50s - loss: -1.6256e+00 - dice_coef: 1.6256 - val_loss: -1.3213e+00 - val_dice_coef: 1.3213
Epoch 549/1000
50s - loss: -1.6373e+0